<a href="https://colab.research.google.com/github/olegsh60/fine-tuning/blob/main/%D0%9F%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BD%D0%B8%D0%B5_RAG_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_LLM_Qwen_2_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ответы на вопросы с помощью LLM модели Qwen 7B + RAG

Устанавливаем общие библиотеки

In [ ]:
# Библиотеки Langchain для взаимодействия с LLM
!pip install -U langchain langchain-community langchain-huggingface -q
# Библиотеки для вектороной DB FAISS и загрузки PDF
!pip install faiss-cpu pypdf -q
# Следующие три строки - установка библиотеки unsloth для использования квантованных LLM.
# Для Colab процесс имеет специфику по версиям библиотек. Если запускать не в Colab, то просто:
# !pip install unsloth
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo -q
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth -q

##Алгоритм создания RAG системы
1). Document Loading

In [ ]:
# Загружаем файл "ГОСТ Р 56939-2016 - безопасность ПО.pdf" для RAG, предварительно его нужно загрузить в Colab в раздел в левой пенели "Files"
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('./ГОСТ Р 56939-2016 - безопасность ПО.pdf')
documents = loader.load()
print("Число страниц: ", len(documents))

Число страниц:  33


2). Text Splitting

In [ ]:
# Определите Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)

split_documents = splitter.split_documents(documents)

print("Число чанков: ", len(split_documents))


Число чанков:  103


3). Embedding

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

#  cuda - Use the GPU (if available)
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cuda"}
)

4). Storage



In [ ]:
from langchain.vectorstores import FAISS

db_embed = FAISS.from_documents(split_documents, hf_embeddings_model)
db_embed.save_local("faiss_db")

5). Retrieval

In [ ]:
# Используем векторноое хранилище и его методов для получения документов
retriever = db_embed.as_retriever(
    search_type="similarity",  # тип поиска похожих документов
    k=4,  # количество возвращаемых документов (Default: 4)
    score_threshold=None,  # минимальный порог для поиска "similarity_score_threshold"
)


## Работа RAG системы


Импорт необходимых обьектов

In [ ]:
from unsloth import FastLanguageModel
from transformers import pipeline
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from langchain.schema import StrOutputParser

<ipython-input-12-22e63ddc0aef>:1: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


##Будем использовать Qwen 2.5 с 7B параметрами квантованную  в 4bit
https://huggingface.co/unsloth/Qwen2.5-7B-bnb-4bit


In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)


==((====))==  Unsloth 2025.4.4: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

##Pipeline для формирования ответа LLM

In [ ]:

terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids(".")]

pipe = pipeline('text-generation',
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=150, # обьем возвращаемых токенов
                repetition_penalty=1.2, # штрав за повторы в ответе немного увеличим от дефортной 1
                temperature=0.7, # Креативность уменьшим от дефортной 1 чтобы получать более точные ответы
                eos_token_id=terminators)

HF_model = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


##Формируем шаблон для промпта

In [ ]:
template = """
Отвечай на вопрос только используя следующий контекст:
{context}
Если ответа нет в контексте, то ответь: я не знаю!
Question: {question}
"""

# Определение шаблона промпта
prompt_template = PromptTemplate.from_template(template)

# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

##Запускаем RAG, очищаем ответ

In [ ]:
query = "Что такое безопасное программное обеспечение?"
context = retriever.get_relevant_documents(query)
context = format_docs(context)
response = HF_model.invoke(prompt_template.format(question=query, context=context))

output_parser = StrOutputParser()
string_result = output_parser.parse(response)

print(string_result)

<ipython-input-16-141ea6567c23>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = retriever.get_relevant_documents(query)



Отвечай на вопрос только используя следующий контекст:
безопасного программного обеспечения

3.2 безопасное программное обеспечение: Программное 
обеспечение, разработанное с использованием совокупности 
мер, направленных на предотвращение появления и 
устранение уязвимостей программы. 
3.3 динамический анализ кода программы: Вид работ по 
инструментальному исследованию программы, основанный на 
анализе кода программы в режиме непосредственного 
исполнения (функционирования) кода. 
3.4 документация разработчика программного 
обеспечения:  Совокупность программных документов,

обеспечения 
5.8.1 Меры по разработке безопасного программного 
обеспечения, подлежащие реализации

и организационные меры, обеспечивающие защиту элементов 
конфигурации от угроз безопасности информации, связанных с 
нарушением конфиденциальности, целостности и доступности. 
5.8.3 Требования к реализации мер по разработке 
безопасного программного обеспечения 
5.8.3.1 Разработчик ПО должен определить элементы 
ко

Пример вопроса на тему не из документа

In [ ]:
# Генерация ответа не по контексту
query = "Какое расстояние до луны?"

context = retriever.get_relevant_documents(query)
context = format_docs(context)
response = HF_model.invoke(prompt_template.format(question=query, context=context))

output_parser = StrOutputParser()
string_result = output_parser.parse(response)

print(string_result)


Отвечай на вопрос только используя следующий контекст:
стандарта документация разработчика ПО должна содержать: 
- сведения о периодичности проведения поиска 
уязвимостей программы; 
- план поиска уязвимостей, описание выполняемых тестов, 
инструментальных средств и общедоступных источников 
информации, используемых при проведении поиска 
уязвимостей программы; 
- отчеты, содержащие список выявленных уязвимостей 
программы (при выявлении), описание действий, направленных 
на их устранение, либо обоснование невозможности или

- описание методов приема и обработки сообщений от 
пользователей об ошибках ПО и уязвимостях программы; 
- описание методов доведения до пользователей 
информации об уязвимостях программы и рекомендаций по их 
устранению, в том числе путем обновления ПО; 
- список выявленных ошибок ПО и уязвимостей программы 
и описание действий, направленных на их устранение, либо 
обоснование невозможности или отсутствия необходимости в 
их устранении. 
Разработчику ПО следует 

Пример запроса к чистой модели без использования RAG

In [ ]:
# Определение шаблона промпта
prompt_template_short = PromptTemplate.from_template("Вопрос: {question}")

# Генерация ответа
query = "Что такое безопасное программное обеспечение?"
response = HF_model.invoke(prompt_template_short.format(question=query))

print(response)

Вопрос: Что такое безопасное программное обеспечение? Ответ: Программное обеспечение, которое не содержит уязвимостей и других ошибок.


Пример вопроса к чистой модели на тему не из документа

In [ ]:
# Определение шаблона промпта
prompt_template_short = PromptTemplate.from_template("Вопрос: {question}")

# Генерация ответа
query = "Какое расстяние до луны?"
response = HF_model.invoke(prompt_template_short.format(question=query))

print(response)

Вопрос: Какое расстяние до луны? Ответ: 384000 километров.
